Step 1) Load the precinct Data

In [1]:
import pandas as pd
from shapely.geometry import Polygon,shape,MultiPolygon
import shapefile as shp
import ast
from collections import deque
import random


In [ ]:
# load in data, and get the min and max district population count for each NJ district
NUM_DISTRICTS=12

nj_precinct_data = pd.read_csv('precinct-data-congress-nj.csv')
total_population=int(nj_precinct_data["Total_2020_Total"].sum())
district_population=int(total_population/NUM_DISTRICTS)

min_pop, max_pop = int(district_population * .95), int(district_population * 1.05)

shpfile = 'nj_vtd_2020_bound.shp'
dbffile = 'nj_vtd_2020_bound.dbf'
shxfile = 'nj_vtd_2020_bound.shx'
prjfile = 'nj_vtd_2020_bound.prj'
cpgfile = 'nj_vtd_2020_bound.cpg'

sf = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile, prj=prjfile, cpg=cpgfile)
nj_precinct_boundaries = {}
for sr in sf.iterShapeRecords():
    geom = sr.shape
    rec = sr.record
    nj_precinct_boundaries[str(rec[3])] = geom  

nj_contiguity_df = pd.read_csv('Contiguity_nj.csv', header=None)
nj_contiguity_df.columns = ['Precinct', 'Neighbors']
nj_contiguity = {}
for index, row in nj_contiguity_df.iterrows():
    neighbors = ast.literal_eval(row['Neighbors'])
    if neighbors and isinstance(neighbors[0], (int, float)):
        nj_contiguity[row['Precinct']] = [str(n) for n in neighbors]
    else:
        nj_contiguity[row['Precinct']] = neighbors

Starting Flood Fill...


In [77]:
# setup floodfill

floodfill_map=nj_precinct_data.copy()
floodfill_map['District']=0
seed_precincts = {}

for index, row in floodfill_map.iterrows():
    geoid = row['GEOID20']
    s = shape(nj_precinct_boundaries[geoid])
    centroid = s.centroid
   
    district_num = 0
    if centroid.x <= -74.72:
        if centroid.y <= 39.3: district_num = 1
        elif centroid.y <= 39.7: district_num = 3
        elif centroid.y <= 40.1: district_num = 5
        elif centroid.y <= 40.5: district_num = 7
        elif centroid.y <= 40.9: district_num = 9
        else: district_num = 11
    else:
        if centroid.y <= 39.3: district_num = 2
        elif centroid.y <= 39.7: district_num = 4
        elif centroid.y <= 40.1: district_num = 6
        elif centroid.y <= 40.5: district_num = 8
        elif centroid.y <= 40.9: district_num = 10
        else: district_num = 12

    if district_num != 0 and district_num not in seed_precincts:
        seed_precincts[district_num] = geoid

    if len(seed_precincts) == NUM_DISTRICTS:
        break



In [78]:
# implement floodfill
# Create lookup dictionaries for faster access
district_map = {row['GEOID20']: 0 for _, row in nj_precinct_data.iterrows()}
population_map = {row['GEOID20']: row['Total_2020_Total'] for _, row in nj_precinct_data.iterrows()}

district_current_pop = {i: 0 for i in range(1, NUM_DISTRICTS + 1)}
district_queues = {i: deque() for i in range(1, NUM_DISTRICTS + 1)}

for district_num, geoid in seed_precincts.items():
    district_map[geoid] = district_num
    district_current_pop[district_num] += population_map[geoid]
    district_queues[district_num].append(geoid)

# Also update the dataframe
floodfill_map = nj_precinct_data.copy()
floodfill_map['District'] = floodfill_map['GEOID20'].map(district_map)

assigned_precincts = set(seed_precincts.values())
unassigned_count = len(floodfill_map) - len(assigned_precincts)

# Helper function to reactivate a district by finding all frontier precincts
def reactivate_district(district_num):
    """Find all precincts in this district that have unassigned neighbors and add them to the queue."""
    district_precincts = [geoid for geoid, d in district_map.items() if d == district_num]
    for precinct in district_precincts:
        # Check if this precinct has unassigned neighbors
        has_unassigned = any(
            district_map.get(neighbor, 0) == 0
            for neighbor in nj_contiguity.get(precinct, [])
        )
        if has_unassigned and precinct not in district_queues[district_num]:
            district_queues[district_num].append(precinct)

# flood fill
iterations_without_progress = 0
max_stagnant_iterations = 1000
iteration_count = 0
print_interval = 10000

while unassigned_count > 0:
    iteration_count += 1
    
    # Print progress every 10,000 iterations
    if iteration_count % print_interval == 0:
        print(f"\n--- Progress after {iteration_count:,} iterations ---")
        print(f"Unassigned precincts: {unassigned_count}")
        for d in range(1, NUM_DISTRICTS + 1):
            print(f"District {d}: {district_current_pop[d]:,} (min: {min_pop:,}, max: {max_pop:,})")
        print(f"Total assigned: {sum(district_current_pop.values()):,}\n")
    # Try to reactivate districts with empty queues
    for d in range(1, NUM_DISTRICTS + 1):
        if not district_queues[d] and district_current_pop[d] < max_pop:
            reactivate_district(d)
    
    growable_districts = [
        d for d in range(1, NUM_DISTRICTS + 1)
        if district_queues[d] and district_current_pop[d] < max_pop
    ]
    
    if not growable_districts:
        # Try one more time to reactivate all districts
        for d in range(1, NUM_DISTRICTS + 1):
            if district_current_pop[d] < max_pop:
                reactivate_district(d)
        growable_districts = [
            d for d in range(1, NUM_DISTRICTS + 1)
            if district_queues[d] and district_current_pop[d] < max_pop
        ]
        
        if not growable_districts:
            print(f"Cannot grow further, but still {unassigned_count} unassigned precincts left.")
            break

    # sort by population, get district with smallest population, get frontier precinct, and get unassigned neighbors
    growable_districts.sort(key=lambda d: district_current_pop[d])
    selected_district = growable_districts[0]
    frontier_precinct = district_queues[selected_district].popleft()
    unassigned_neighbors = [
        neighbor for neighbor in nj_contiguity.get(frontier_precinct, [])
        if district_map.get(neighbor, 0) == 0
    ]

    if not unassigned_neighbors:
        # Check if this precinct might get neighbors later - re-add to end of queue
        # But limit how many times we check the same precinct
        if frontier_precinct not in list(district_queues[selected_district])[-10:]:
            district_queues[selected_district].append(frontier_precinct)
        iterations_without_progress += 1
        if iterations_without_progress > max_stagnant_iterations:
            # Force reactivation of all districts
            for d in range(1, NUM_DISTRICTS + 1):
                if district_current_pop[d] < max_pop:
                    reactivate_district(d)
            iterations_without_progress = 0
        continue

    iterations_without_progress = 0

    # get neighbors that wont exceed max population
    possible_neighbors = [
        n for n in unassigned_neighbors
        if (population_map[n] + district_current_pop[selected_district]) <= max_pop
    ]

    if not possible_neighbors:
        # Re-add frontier to queue since it has unassigned neighbors (even if we can't add them yet)
        district_queues[selected_district].append(frontier_precinct)
        continue

    next_precinct = random.choice(possible_neighbors)
    district_map[next_precinct] = selected_district
    pop_add = population_map[next_precinct]
    district_current_pop[selected_district] += pop_add
    district_queues[selected_district].append(next_precinct)
    unassigned_count -= 1
    
    # Check if frontier still has unassigned neighbors
    still_has_unassigned = any(
        district_map.get(n, 0) == 0
        for n in nj_contiguity.get(frontier_precinct, [])
    )

    if still_has_unassigned:
        district_queues[selected_district].append(frontier_precinct)

# Update dataframe with final assignments
floodfill_map['District'] = floodfill_map['GEOID20'].map(district_map)

print("District populations after flood fill:")
for d in range(1, NUM_DISTRICTS + 1):
    print(f"District {d}: {district_current_pop[d]} (min: {min_pop}, max: {max_pop})")

print(f"\nUnassigned precincts remaining: {unassigned_count}")
print(f"Total assigned: {sum(district_current_pop.values())}")

floodfill_map[['GEOID20', 'District']].to_csv('fair_nj_map.csv', index=False)


--- Progress after 10,000 iterations ---
Unassigned precincts: 5297
District 1: 123,228 (min: 735,377, max: 812,786)
District 2: 124,702 (min: 735,377, max: 812,786)
District 3: 123,969 (min: 735,377, max: 812,786)
District 4: 125,778 (min: 735,377, max: 812,786)
District 5: 124,935 (min: 735,377, max: 812,786)
District 6: 123,674 (min: 735,377, max: 812,786)
District 7: 123,433 (min: 735,377, max: 812,786)
District 8: 123,539 (min: 735,377, max: 812,786)
District 9: 123,572 (min: 735,377, max: 812,786)
District 10: 123,259 (min: 735,377, max: 812,786)
District 11: 123,314 (min: 735,377, max: 812,786)
District 12: 124,121 (min: 735,377, max: 812,786)
Total assigned: 1,487,524


--- Progress after 20,000 iterations ---
Unassigned precincts: 5297
District 1: 123,228 (min: 735,377, max: 812,786)
District 2: 124,702 (min: 735,377, max: 812,786)
District 3: 123,969 (min: 735,377, max: 812,786)
District 4: 125,778 (min: 735,377, max: 812,786)
District 5: 124,935 (min: 735,377, max: 812,786)

KeyboardInterrupt: 